In [1]:
import random
from firebase_admin import initialize_app, credentials, db

In [2]:
cred = credentials.Certificate('creds.json')

In [3]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [ ]:
players = {'Simon': 'gruft', 'Dave': 'clutch', 'Heiko': 'milch', 'Niko': 'wow'}

In [14]:
turn_ref = db.reference("turn")
board_ref = db.reference("board")
stack_ref = db.reference("stack")
players_ref = db.reference("players")

In [13]:
turn_ref.get()

{'currentTurn': 1,
 'longestRoadCount': 5,
 'longestRoadPlayer': 1,
 'maxKnightCount': 2,
 'names': ['', 'Ernie', 'Bert']}

In [32]:
players_ref.get()

{'clutch': {'name': 'Dave', 'number': 2},
 'excel': {'name': 'Niko', 'number': 4},
 'milch': {'name': 'Heiko', 'number': 3},
 'rave': {'name': 'Simon', 'number': 1}}

In [209]:
h = "Holz"
l = "Lehm"
g = "Getreide"
w = "Wolle"
e = "Erz"
x = "Wüste"

resources = [h, l, g, w, e]
resource_order = {r: i for i, r in enumerate(resources)}

In [37]:
# init board
tiles = [[H, W, W], [G, E, G, H], [H, L, X, E, G], [G, E, H, W], [L, W, L]]
rolls = [[6, 3, 8], [2, 4, 5, 10], [5, 9, None, 6, 9], [10, 11, 3, 12], [8, 4, 11]]
ports = ['*', W, '*', E, '*', G, L, '*', H]

board = [[{'id': row * 100 + col} for col in range(21)] for row in range(11)]

for r, ts in enumerate(tiles):
    row = 1 + (r * 2)
    pad = (1 + abs(r - 2)) * 2 
    for c, tile in enumerate(ts):
        col = pad + c * 4
        board[row][col]['tile'] = tile
        roll = rolls[r][c]
        if roll is not None:
            board[row][col]['roll'] = roll
        else:
            board[row][col]['bandit'] = True

board[0][5]['port'] = ports[0]
board[0][5]['face'] = 'NW'
board[0][11]['port'] = ports[1]
board[0][11]['face'] = 'NE'
board[2][17]['port'] = ports[2]
board[2][17]['face'] = 'NE'
board[3][2]['port'] = ports[3]
board[3][2]['face'] = 'W'
board[5][20]['port'] = ports[4]
board[5][20]['face'] = 'E'
board[7][2]['port'] = ports[5]
board[7][2]['face'] = 'W'
board[8][17]['port'] = ports[6]
board[8][17]['face'] = 'SE'
board[10][5]['port'] = ports[7]
board[10][5]['face'] = 'SW'
board[10][11]['port'] = ports[8]
board[10][11]['face'] = 'SE'

board_ref.set(board)

In [38]:
# set players
player_refs = [players_ref.child(k) for k in players.values()]
res_refs = [r.child('resources') for r in player_refs]
card_refs = [r.child('cards') for r in player_refs]
player_count = len(players)

In [35]:
# init players
player_data = {k: {"index": i} for i, (n, k) in enumerate(players.items())}
players_ref.set(player_data) 

In [9]:
# init cards
stack = ["Ritter"] * 14 + ["Monopol", "Strassenbau", "Erfindung"] * 2
stack += ["Bibliothek", "Marktplatz", "Rathaus", "Kirche", "Universität"]
random.shuffle(stack)
stack_ref.set(stack)

In [39]:
# init turn
names = [""] + list(players)
first_index = random.randrange(player_count)
turn_ref.set({'names': names, 'index': first_index, 'maxKnightCount': 2})

In [106]:
current

1

In [108]:
current_index = SI

In [ ]:
current_index = first_index + 1

In [ ]:
current_index += 1

In [67]:
current_index -= 1

In [73]:
# next placement
turn_ref.update({'index': current_index})

In [253]:
# placement
P = S
board = board_ref.get()
#board[8][2]['town'] = P+1
#board[4][16]['city'] = True
board[4][5]['road'] = P+1
board_ref.set(board)

In [249]:
# next turn
current_index = (current_index + 1) % player_count
roll = random.randint(1, 6) + random.randint(1, 6)
turn_ref.update({'index': current_index, 'roll': roll})

In [12]:
# set longest road
turn_ref.update({'longestRoadPlayer': 1, 'longestRoadCount': 5})

In [88]:
# init resource
resources = sorted([H, W, E], key=lambda r: resource_order[r])
res_refs[3].set(resources)

In [210]:
s = player_refs[0]
d = player_refs[1]
h = player_refs[2]
n = player_refs[3]

In [244]:
# add resources
P = N
resources = res_refs[P].get() or []
resources += [l, l]
resources = sorted(resources, key=lambda r: resource_order[r])
res_refs[P].set(resources)
resources

['Holz',
 'Holz',
 'Lehm',
 'Lehm',
 'Getreide',
 'Wolle',
 'Wolle',
 'Erz',
 'Erz',
 'Erz',
 'Erz']

In [254]:
# pay resources
P = S
#X = [h, l]
#X = [h, l, g, w]
#X = [g, g, e, e, e]
X = [g, w, e]
#X = [g, g, g, g]
resources = res_refs[P].get()
for resource in X:
    resources.remove(resource)
res_refs[P].set(resources)
resources

['Holz', 'Wolle']

In [250]:
players_ref.get()

{'clutch': {'name': 'Dave',
  'number': 2,
  'resources': ['Holz',
   'Holz',
   'Getreide',
   'Getreide',
   'Getreide',
   'Getreide',
   'Wolle',
   'Erz']},
 'excel': {'name': 'Niko', 'number': 4},
 'gruft': {'name': 'Simon',
  'number': 1,
  'resources': ['Holz', 'Holz', 'Lehm', 'Getreide', 'Wolle', 'Wolle', 'Erz']},
 'milch': {'name': 'Heiko', 'number': 3, 'resources': ['Wolle', 'Erz']},
 'wow': {'resources': ['Wolle', 'Erz', 'Erz', 'Erz', 'Erz']}}

In [195]:
# bandit
board = board_ref.get()
board[5][2]['bandit'] = True
board[3][4]['bandit'] = None
board_ref.set(board)

In [196]:
# bandit steal
P = SI
resources = res_refs[P].get()
random.choice(resources)

'Erz'

In [ ]:
# check bandit

for player_num, player_ref in self.player_refs:

    player = player_ref.get()
    over_count = len(player['resources']) - 7
    if over_count > 0:
        losses = random.sample(player['resources'], over_count)
        self.update_player(player_num, losses=losses)

In [ ]:
def road(alias):
P = S
X = [h, l]
#X = [h, l, g, w]
#X = [g, g, e, e, e]
#X = [g, w, e]
#X = [h]
resources = ref.get()
for resource in X:
    resources.remove(resource)
ref.set(resources)

P = N
board = board_ref.get()
#board[8][2]['town'] = P+1
#board[4][16]['city'] = True
board[6][17]['road'] = P+1
board_ref.set(board)

In [255]:
# add card
P = S
cards = card_refs[P].get() or []
cards.append("Strassenbau")
card_refs[P].set(cards)
cards

['Strassenbau']

In [9]:
player['cards'].remove(card)
player['cards'] += gained_cards

if "Ritter (gespielt)" in gained_cards:
    knight_count = player['cards'].count("Ritter (gespielt)")
    turn = turn_ref.get()
    if knight_count > turn['maxKnightPlayer']:
        turn['maxKnightCount'] = knight_count
        turn['maxKnightPlayer'] = player_num
        turn_ref.set(turn)

player_refs[p].update(player)




def steal_resource(self, dest_num, orig_num):

    orig = self.player_refs[orig_num].get()
    if len(orig['resources']) == 0:
        return
    resource = random.choice(orig['resources'])
    self.update_player(orig_num, losses=[resource])
    self.update_player(dest_num, gains=[resource])


def check_bandit(self):

    # TODO select which resources?
    for player_num, player_ref in self.player_refs:

        player = player_ref.get()
        over_count = len(player['resources']) - 7
        if over_count > 0:
            losses = random.sample(player['resources'], over_count)
            self.update_player(player_num, losses=losses)


def buy(self, player_num, roads=0, towns=0, cities=0, cards=0):

    costs = [H, L] * roads + [H, L, G, W] * towns + [G, G, E, E, E] * cities + [G, W, E] * cards

    drawn_cards = None
    if cards > 0:
        stack = self.stack_ref.get()
        drawn_cards = stack[:cards]
        self.stack_ref.set(stack[cards:])

    self.update_player(player_num, losses=costs, gained_cards=drawn_cards)


def move_bandit(self, bid):

    board = self.board_ref.get()
    prev_cell = [c for r in board for c in r if 'bandit' in c][0]

    self.set_cell_attribute(prev_cell['id'], 'bandit', None)
    self.set_cell_attribute(bid, 'bandit', True)


def build_road(self, player_num, bid):

    self.set_cell_attribute(bid, 'road', player_num)


def build_town(self, player_num, bid):

    self.set_cell_attribute(bid, 'town', player_num)


def upgrade_town(self, bid):

    self.set_cell_attribute(bid, 'city', True)


def play_card(self, player_num, card, param=None):

    gains = None
    gained_cards = None

    if card == 'Ritter':
        gained_cards = ["Ritter (gespielt)"]

    elif card == 'Strassenbau':
        # TODO: grant 2 streets?
        pass

    elif card == 'Monopol':

        gains = []
        for other_num, player_ref in enumerate(self.player_refs):
            if other_num == player_num:
                continue
            other = self.player_refs[other_num].get()
            losses = [r for r in other['resources'] if r == param]
            gains += losses
            self.update_player(other_num, losses=losses)

    elif card == 'Erfindung':

        gains = params

    else:
        return

    self.update_player(player_num, gains=param, gained_cars=gained_cards, lost_cards=[card])

In [ ]:
random.randint(1, 6) + random.randint(1, 6)

Fix:
- backend
    - trade functions
    - bandit loss communicate
    - block for every buy
- frontend
    - Gelb Kontrast!